In [3]:
import gtfs_functions as gtfs
import pandas as pd

In [4]:
apc_df =  pd.read_parquet('./carta-apc.parquet/2021_10_24_gtfs_days', engine='pyarrow').sort_values('transit_date').reset_index(drop=True)
apc_df = apc_df[apc_df['gtfs_trip_id']>='204888020']
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs('./20220111.zip', busiest_date=False)

In [7]:
apc_df = apc_df[apc_df.transit_date=='2022-01-11']

In [14]:
unique_trp = apc_df.trip_id.unique()

In [75]:
import pandas as pd
import json
df1 = pd.read_csv("./2021-10-24/trips.txt", sep=",")
df2 = pd.read_csv("./2021-10-24/stop_times.txt", sep=",")
df3 = pd.read_csv("./2021-10-24/stops.txt", sep=",")
df4 = pd.read_csv("./2021-10-24/shapes.txt", sep=",")
df5 = pd.read_csv("./2021-10-24/routes.txt", sep=",")

trips = unique_trp
trips = list(map(int, trips))

df1_fil  = pd.DataFrame(columns = df1.columns)
df2_fil  = pd.DataFrame(columns = df2.columns)

indices = []
for trip in trips:
    for index, row in df1.iterrows():       
        if trip == int(row['trip_id']/1000):
            
            indices.append(index)
            df1_fil.append(row, ignore_index=True)
            break

df1_fil = df1.iloc[indices].reset_index(drop=True)

#removing route 10C
df1_fil = df1_fil[df1_fil.route_id != '10C'].reset_index(drop=True)

trips = df1_fil.trip_id.unique()


In [90]:
df2_fil = df1_fil[['trip_id']].merge(df2, on ='trip_id', how='left', indicator=True)
df2_fil = df2_fil[df2_fil._merge == 'both'].drop('_merge', axis=1).sort_values(['arrival_time', 'trip_id']).reset_index(drop=True)

In [56]:
df3_fil = df2_fil.drop_duplicates(subset=['stop_id'])[['stop_id']].merge(df3, on ='stop_id', how='left', indicator=True)
df3_fil = df3_fil[df3_fil._merge == 'both'].drop('_merge', axis=1).reset_index(drop=True)

In [68]:
df4_fil = df1_fil.drop_duplicates(subset=['shape_id'])[['shape_id']].merge(df4, on ='shape_id', how='left', indicator=True)
df4_fil = df4_fil[df4_fil._merge == 'both'].drop('_merge', axis=1).reset_index(drop=True)

In [80]:
df5_fil = df1_fil.drop_duplicates(subset=['route_id'])[['route_id']].merge(df5, on ='route_id', how='left', indicator=True)
df5_fil = df5_fil[df5_fil._merge == 'both'].drop('_merge', axis=1).sort_values('route_id').reset_index(drop=True)

In [92]:
#write to txt
df1_fil.to_csv(r'./20220111-APCtrips/trips.txt', header=True, index=None, sep=',', mode='a')
df2_fil.to_csv(r'./20220111-APCtrips/stop_times.txt', header=True, index=None, sep=',', mode='a')
df3_fil.to_csv(r'./20220111-APCtrips/stops.txt', header=True, index=None, sep=',', mode='a')
df4_fil.to_csv(r'./20220111-APCtrips/shapes.txt', header=True, index=None, sep=',', mode='a')
df5_fil.to_csv(r'./20220111-APCtrips/routes.txt', header=True, index=None, sep=',', mode='a')